In [1]:
import wikipedia

actor_name = "Leonardo DiCaprio"
try:
    page = wikipedia.page(actor_name).content
    words = page.split()
    
    # if len(words) >= 500:
    #     page = ' '.join(words[:600]) # Extract the first 600 words
    with open('wikipedia.txt', 'w', encoding='utf-8') as file:
        file.write(page)
    # else:
    #     print(f"The Wikipedia page for {actor_name} has less than 500 words. Please choose a different entry.")
except Exception:
    print(f"An unexpected error occurred")

In [2]:
import wikipedia

actor_name = "Leonardo DiCaprio"
try:
    page = wikipedia.page(actor_name).content
    words = page.split()
    
    # if len(words) >= 500:
    #     page = ' '.join(words[:600]) # Extract the first 600 words
    with open('wikipedia.txt', 'w', encoding='utf-8') as file:
        file.write(page)
    # else:
    #     print(f"The Wikipedia page for {actor_name} has less than 500 words. Please choose a different entry.")
except Exception:
    print(f"An unexpected error occurred")

In [5]:
import requests

def extract_wikipedia_pages(start_letter='H', limit=10):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "allpages",
        "apnamespace": 0,  # Only get pages from the main namespace
        "apprefix": start_letter,  # Get pages starting with the specified letter
        "aplimit": limit,  # Limit the number of results
        "format": "json"
    }

    page_titles = []
    
    while True:
        response = requests.get(url, params=params)
        data = response.json()
        
        # Extract the page titles from the response
        for page in data['query']['allpages']:
            page_titles.append(page['title'])

        # Check if there are more pages to fetch
        if 'continue' in data:
            params['continue'] = data['continue']['continue']
        else:
            break

    return page_titles

def extract_categories(page_titles):
    categories = {}
    url = "https://en.wikipedia.org/w/api.php"

    for title in page_titles:
        params = {
            "action": "query",
            "titles": title,
            "prop": "categories",
            "format": "json"
        }
        
        response = requests.get(url, params=params)
        data = response.json()
        page = next(iter(data['query']['pages'].values()))  # Get the first page

        if 'categories' in page:
            category_names = [cat['title'] for cat in page['categories']]
            categories[title] = category_names
        else:
            categories[title] = []

    return categories

def save_to_txt(file_name, page_data):
    with open(file_name, 'w', encoding='utf-8') as file:
        for title, cats in page_data.items():
            cats_str = ', '.join(cats) if cats else 'No categories'
            file.write(f"Title: {title}\nCategories: {cats_str}\n\n")

# Example Usage
page_titles = extract_wikipedia_pages(start_letter='H', limit=500)
categories = extract_categories(page_titles)

# Save the titles and categories to a text file
save_to_txt('wikipedia_pages_starting_with_H.txt', categories)

print(f"Extracted {len(categories)} pages starting with 'H' to 'wikipedia_pages_starting_with_H.txt'")
